# Load packages

In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
import math

# Read data

In [4]:
dataset = pd.read_csv('/resources/data/samples/DataScienceProjekt17/train.csv')
kaggle_test = pd.read_csv('/resources/data/samples/DataScienceProjekt17/test.csv')

# Data cleaning and preprocess

In [5]:
# delete ID
del dataset['Id']
# convert to 'real' Nan
dataset.fillna('NaN', inplace=True)

# Linear Regression Model

In [17]:
x=np.log(dataset[['LotArea']].copy())
x['Constant']=np.ones(len(x))
x['LotSq']=np.log(dataset[['LotArea']]**2)
tmp=dataset[['MSSubClass']]
tmp=pd.Series(tmp, dtype="category")
x=pd.concat([x,pd.get_dummies(dataset['MSSubClass']),
            pd.get_dummies(dataset['YrSold']),
            pd.get_dummies(dataset['OverallQual']),
   #        pd.get_dummies(dataset['OverallCond']),
            pd.get_dummies(dataset['SaleType']),
            pd.get_dummies(dataset['MSZoning'])], axis=1)
x=x.values
y=dataset['SalePrice'].values

In [18]:
cv_rmsel=0
reg=LinearRegression()
for i in range(10):
    rmsel=0
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)
    reg.fit(x_train,y_train)
    prediction=(reg.predict(x_test))
    for i in range(len(y_test)):
        rmsel += (math.log(y_test[i]+1) - math.log(prediction[i]+1))**2
    rmsel=(rmsel/len(x_test))**0.5
    cv_rmsel += cv_rmsel
cv_rmsel=cv_rmsel/10
print(rmsel)

0.1836097740155874


# Final kaggle prediction and submission

In [8]:
reg=LinearRegression()
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Unknown data filter...

In [10]:
kaggle_test['SaleType']=kaggle_test['SaleType'].replace('NaN','Oth')      # for now, assign NaN to Oth
kaggle_test['MSZoning']=kaggle_test['MSZoning'].replace('NaN','FV')       # for now, assign NaN to something
kaggle_test['MSSubClass']=kaggle_test['MSSubClass'].replace('150','180')  # for now, assign new train class to 180
kaggle_test['MSSubClass']=pd.to_numeric(kaggle_test['MSSubClass'])        # for now, assign NaN to something

Transform test dataset

In [11]:
kaggle_x=np.log(kaggle_test[['LotArea']].copy())
kaggle_x['Constant']=np.ones(len(kaggle_x))
kaggle_x['LotSq']=np.log(kaggle_test[['LotArea']]**2)
kaggle_tmp=kaggle_test[['MSSubClass']]
kaggle_tmp=pd.Series(tmp, dtype="category")
kaggle_x=pd.concat([kaggle_x,pd.get_dummies(kaggle_test['MSSubClass']),
            pd.get_dummies(kaggle_test['YrSold']),
            pd.get_dummies(kaggle_test['OverallQual']),
   #        pd.get_dummies(kaggle_test['OverallCond']),
            pd.get_dummies(kaggle_test['SaleType']),
            pd.get_dummies(kaggle_test['MSZoning'])], axis=1)
kaggle_x=kaggle_x.values

Make prediction and convert to csv

In [12]:
prediction=reg.predict(kaggle_x)
prediction=pd.concat([kaggle_test['Id'],pd.DataFrame(prediction)],axis=1)
prediction.columns=["Id","SalePrice"]
prediction.to_csv('submission.csv',index=False)

In [13]:
prediction.head()

,Id,SalePrice
0,1461,151093.851774
1,1462,179780.052695
2,1463,175544.787705
3,1464,184574.533392
4,1465,247826.868594
